In [ ]:
%load_ext autoreload
%autoreload 2


import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tqdm import tqdm
import glob
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from preprocess import *
from preprocess import remove_outlier_trajectories
from road_network import RoadNetwork


In [ ]:
# load network
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)
network.save(path="../osm_data/sf")

# After saving can be loaded like this:
#network = RoadNetwork()
#network.load("../osm_data/sf")

In [ ]:
# read data 

all_files = glob.glob(os.path.join("../datasets/trajectories/sf/cabdata" , "*.txt"))

data = []

for filename in all_files:
    tdf = pd.read_csv(filename, index_col=None, header=None, delimiter=" ")
    tdf["tax_id"] = filename.split("/")[-1].split(".")[0].split("_")[1]
    data.append(tdf)

df = pd.concat(data, axis=0, ignore_index=True)
df = df.rename(columns={0: "lat", 1: "long", 2: "occupied", 3: "timestamp"})

In [ ]:

import time 
# group for each taxi
rows = []
for _, g in tqdm(df.groupby("tax_id")):
    # group each occupied trajectory
    trajectories_occu = g[g['occupied'] == 1].groupby((g['occupied'] != 1).cumsum())
    # trajectories_nooccu = g[g['occupied'] == 0].groupby((g['occupied'] != 0).cumsum())
    for _, t in trajectories_occu:
        if t.shape[0] < 5:
            continue
        data = t.to_numpy()
        data[:, 0], data[:, 1] = data[:, 1], data[:, 0].copy()
        seq = LineString(data[::-1, :2].astype(np.float32))
        stamps = data[::-1, 3]
        rows.append((seq, stamps - stamps[0]))
    
    # for _, t in trajectories_nooccu:
    #     if t.shape[0] < 5:
    #         continue
    #     data = t.to_numpy()
    #     seq = LineString(data[::-1, :2])
    #     rows.append((seq, data[::-1, 3]))

processed_df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp"])
processed_df

In [ ]:
df.to_csv("../datasets/trajectories/sf/all_gps_points.csv", sep=";", index=False)

In [ ]:
city_bounds = network.bounds
clipped = clip_trajectories(processed_df.copy(), city_bounds, polyline_convert=True)
# df_clipped = filter_min_points(df_clipped, 5)
clipped

In [ ]:
""" 
Correct timestamps
"""

def strictly_increasing(L):
    return all(x+20>=y for x, y in zip(L, L[1:]))


def correct_timestamps(traj, orig_trajs, orig_ts):
    corrected_ts = []
    corrected_traj = []
    idxs = []
    found = False
    for i, g1 in enumerate(traj):
        ridx = 0
        for j, g2 in enumerate(orig_trajs[ridx:]):
            if g1 == g2:
                found = True
                corrected_ts.append(orig_ts[j])
                corrected_traj.append(g1)
                idxs.append(j)
                ridx = j+1
                break
            # if found:
            #     break

    assert len(corrected_traj) == len(corrected_ts)
    # assert strictly_increasing(idxs), (idxs)
    
    return corrected_traj, (np.array(corrected_ts) - corrected_ts[0]).astype(int).tolist()


rows = []
i = 0
orig_polies, orig_ts = processed_df.POLYLINE, processed_df.timestamp
for i, r in tqdm(clipped.iterrows()):
    op = list(orig_polies.loc[r.name].coords)
    ot = orig_ts.loc[r.name]
    if type(r.POLYLINE) == LineString:
        traj = list(r.POLYLINE.coords)
        if len(traj) < 5:
            continue
        ctraj, cts = correct_timestamps(traj, op, ot)
        if len(ctraj) < 5:
            continue
        rows.append([LineString(ctraj), cts])
    else:
        for line in r.POLYLINE:
            traj = list(line.coords)
            if len(traj) < 5:
                continue
            ctraj, cts = correct_timestamps(traj, op, ot)
            if len(ctraj) < 5:
                continue
            rows.append([LineString(ctraj), cts])

df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp"])

In [ ]:
df["id"] = np.arange(1, df.shape[0]+1)
df["timestamp"] = df["timestamp"].astype(str)
df["timestamp"] = df["timestamp"].str.replace("[", "")
df["timestamp"] = df["timestamp"].str.replace("]", "")
# df_clipped["timestamp"] = df["timestamp"].str.replace(" ", ", ")
df.to_csv("../datasets/trajectories/sf/mapped_id_poly_clipped_corrected.csv", sep=";", index=False)

In [ ]:
df = pd.read_csv("../datasets/trajectories/sf/mapped_id_poly_clipped_corrected.csv", sep=";")
df

In [ ]:
list(df.iloc[418135].POLYLINE.coords)

In [ ]:
df["POLYLINE"] = df["POLYLINE"].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="POLYLINE")

## Map Matching

Next we need to map match the trajectories. We use FastMapMatching (https://fmm-wiki.github.io/). For faster map matching, we recommend using the command line programm instead of the python wrapper, which is used in the next cell. 

In [ ]:
network.fmm_trajectorie_mapping(
    network_file="../osm_data/sf/edges.shp",
    input_file="../datasets/trajectories/SF/mapped_id_poly_clipped.csv",
    output_file="../datasets/trajectories/SF/road-segment-mapping.txt"
)

In [ ]:
# preprocess the mapping especially the speed and distance values need to be verified
df = pd.read_csv("../datasets/trajectories/SF/road_segment_map_final.csv", sep=";")
df_prep = remove_outlier_trajectories(df.copy(), min_edges_traversed=3)
df_prep.to_csv("../datasets/trajectories/SF/road_segment_map_final.csv", sep=";")

In [ ]:
df = pd.read_csv("../datasets/trajectories/SF/road_segment_map_final.csv", sep=";")
df = df[df["speed_mean"] * 111000 * 3.6 < 100]
df.to_csv("../datasets/trajectories/SF/road_segment_map_final.csv", sep=";")

In [ ]:
"""
Test of Travel Time Dataset generation
"""
from trajectory import Trajectory

traj = Trajectory("../datasets/trajectories/sf/road_segment_map_final.csv", nrows=1000000)

In [ ]:
temp = pd.read_csv("../datasets/trajectories/sf/road_segment_map_final_corrected_sf.csv", sep=";")

In [ ]:
dft = traj.generate_TTE_datatset()
dft["travel_time"].describe()

In [ ]:
# delete corrupt trajs and save
temp = temp[~temp["id"].isin(dft[dft["travel_time"] <= 10]["id"].values)]
temp.to_csv("../datasets/trajectories/sf/road_segment_map_final.csv", sep=";")

In [ ]:
"""
Generate traj features 
"""
features = traj.generate_speed_features(network)


In [ ]:
features[features["avg_speed"] < 0] = 0

In [ ]:
features.to_csv("../datasets/trajectories/sf/speed_features_unnormalized.csv")

In [ ]:
features.describe()

In [ ]:
df = pd.read_csv("../datasets/trajectories/SF/road_segment_map_final.csv", sep=";")
df